In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
DATE_RANGE = ('04-01-2016', '10-31-2016')

df = pd.read_csv(
    f'./data/city/Toulouse/X_hour_toulouse.csv',
    index_col=0
)
df['date'] = pd.date_range(*DATE_RANGE, freq='1h')[:len(df)]
df.set_index('date', inplace=True)

In [4]:
a = pd.Series([False, True, True, False, True], index=[0, 1, 2, 9, 10])
a[0]

False

In [29]:
def get_interpolated_indices(serie: pd.Series, tolerance: float=1e-3, output_type: str='mask'):

    mask, current_mask = [], []
    for index in range(len(serie)):
        # Si < 2 éléments d'affilé de suite
        if len(current_mask) < 2:
            current_mask.append(serie.index.values[index])
            continue
        
        # On check si la différence est la même entre trois éléments d'affilé de suite
        if np.abs((serie.values[index - 2] - serie.values[index - 1]) - (serie.values[index - 1] - serie.values[index])) < tolerance:
            current_mask.append(serie.index.values[index])
            continue
        
        # Si la différence est constante et la longueur de l'interpolation < 24 heures
        if len(current_mask) < 24 and np.abs(serie.values[index - 2] - serie.values[index - 1]) < tolerance:
            current_mask = []

        if len(current_mask) > 2:
            mask.append(current_mask[1:-1])
        current_mask = [serie.index.values[index - 1], serie.index.values[index]]

    if output_type == 'maks':
        return serie[[index_mask for row in mask for index_mask in row]].index

    if output_type == 'list':
        return mask
    
    raise ValueError('The "output_type" value must be one of the following : "mask", "list".')

In [4]:
def get_interpolated_indices(serie: pd.Series, tolerance: float=1e-9, output_type: str='mask') -> pd.Index | pd.DatetimeIndex:
    interpolated_mask = np.roll((serie.diff().abs() > tolerance) & (serie.diff().diff().abs() < tolerance), -1)

    if output_type == 'mask':
        return serie[interpolated_mask].index
    
    if output_type == 'list':
        serie_mask = pd.Series(interpolated_mask, index=serie.index.values)
        list_mask, current_list = [], []
        for index, bool_value in serie_mask.items():
            if bool_value:
                current_list.append(index)
                continue
            
            if current_list:
                list_mask.append(current_list)
                current_list = []
        return list_mask

    raise ValueError('The "output_type" value must be one of the following : "mask", "list".')

In [5]:
index_column = 0
tmp = df.loc[:].iloc[:, index_column].copy()
# tmp = pd.Series([-2, 1, 2, 3, 84, 56, 98, 5, 4, 3, 2, 92])

index = get_interpolated_indices(tmp, output_type='list')
print(len(index))
index = [u for v in index for u in v]
tmp.loc[index] = np.nan

a = tmp.to_frame()
a['origine'] = df.iloc[:, index_column].copy()
fig = px.line(a).show()

40
